# 03 U.S. Bank Aggregates — Cleaning & Export

Goal: Clean banking sector metrics, enforce numeric types, create Power BI keys (`date_key`, `geo_key`), export `bank_aggregates_cleaned.csv`.

Guardrails: work on copies, no inplace, explicit conversions, validate early, conservative cleaning, document decisions.


What: Import libraries and constants.
Why: Keep config explicit and reproducible.


In [1]:
# Helpers: drop unnamed and sanitize
import re

def drop_unnamed(df):
    return df.loc[:, ~df.columns.astype(str).str.match(r'^Unnamed')]

def sanitize_columns(df):
    def _san(c):
        c = str(c).strip().lower()
        c = re.sub(r"\s+", "_", c)
        c = re.sub(r"[^a-z0-9_]+", "_", c)
        c = re.sub(r"_+", "_", c).strip('_')
        return c
    df = df.copy()
    df.columns = [_san(c) for c in df.columns]
    return df


In [2]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 160)

ANALYSIS_START_YEAR = 2000
ANALYSIS_END_YEAR = 2024
USA_GEO_KEY = "USA"

ROOT = os.path.abspath(os.path.join(os.getcwd(), "..")) if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()
RAW = os.path.join(ROOT, "original_data")
CLEAN = os.path.join(ROOT, "data", "cleaned")
os.makedirs(CLEAN, exist_ok=True)


What: Load Excel into a working copy and inspect.
Why: Validate structure before conversions.


In [3]:
# Correct date parsing (compatibility): avoid `.dt.isna()` on Series
# Recompute mask_scope and df_clean with safe `.isna()`
if 's_date' in locals():
    if s_date.isna().mean() > 0.5:
        s_date = pd.to_datetime(df[DATE_COL].astype('Int64').astype(str) + '-01-01', errors='coerce')
    mask_scope = (s_date.dt.year >= ANALYSIS_START_YEAR) & (s_date.dt.year <= ANALYSIS_END_YEAR)
    print('Rows in scope (recomputed):', mask_scope.sum(), '/', len(df))
    df_clean = df.copy()
    df_clean[DATE_COL] = s_date


In [4]:
# Load (sheet selection if needed)
raw_path = os.path.join(RAW, "bank_aggregates.xlsx")
df_raw = pd.read_excel(raw_path)
df_raw = drop_unnamed(df_raw)
df = df_raw.copy()
print("Shape:", df.shape)
print(df.head())
print(df.info())


Shape: (113, 38)
        Date  Bank Sector /GDP  Number of Commercial Banks  Cash  Invt Securities  Loans  Trading Assets  All Other Assets  Int-Bearing Deps  \
0 1991-01-01          0.546093                       12124   NaN              NaN    NaN             NaN               NaN               NaN   
1 1991-04-01          0.543184                       12034   NaN              NaN    NaN             NaN               NaN               NaN   
2 1991-07-01          0.536502                       11952   NaN              NaN    NaN             NaN               NaN               NaN   
3 1991-10-01          0.542188                       11812   NaN              NaN    NaN             NaN               NaN               NaN   
4 1992-01-01          0.534436                       11700   NaN              NaN    NaN             NaN               NaN               NaN   

   Nonint-Bearing Deps  Fed Funds and Repo  Trading Liabs  Oth Borr Money  Other Liabs  RE Loans  C&I Loans  Consumer 

What: Detect date/period column; coerce dates and numerics; scope years.
Why: Ensure correct time grain and numeric types.


In [5]:
# Detect date/period column and coerce numerics
possible_date_cols = [c for c in df.columns if 'date' in c.lower() or 'period' in c.lower() or 'year' in c.lower() or 'month' in c.lower()]
print('Possible date/period columns:', possible_date_cols)

# Choose first for scaffold; adjust if needed
DATE_COL = possible_date_cols[0] if possible_date_cols else None
assert DATE_COL is not None, 'No date/period column found; set DATE_COL manually.'

# Parse date; if year-only, default to Jan-01
s_date = pd.to_datetime(df[DATE_COL], errors='coerce')
if s_date.isna().mean() > 0.5:
    # Try treating as year
    s_date = pd.to_datetime(df[DATE_COL].astype('Int64').astype(str) + '-01-01', errors='coerce')

mask_scope = (s_date.dt.year >= ANALYSIS_START_YEAR) & (s_date.dt.year <= ANALYSIS_END_YEAR)
print('Rows in scope:', mask_scope.sum(), '/', len(df))

df_clean = df.copy()
df_clean[DATE_COL] = s_date

# Coerce numerics
for c in df_clean.columns:
    if c != DATE_COL:
        df_clean[c] = pd.to_numeric(df_clean[c], errors='coerce')


Possible date/period columns: ['Date']
Rows in scope: 77 / 113


What: Create keys and export fact CSV.
Why: Make dataset immediately consumable in Power BI.


In [6]:
# Keys and export
fact = df_clean.loc[mask_scope].copy()
# drop rows with null date
fact = fact.loc[fact[DATE_COL].notna()].copy()
# drop corrupt rows where all numeric measures are NaN
numeric_cols = [c for c in fact.columns if c != DATE_COL]
# coerce numeric again just to be safe
for c in numeric_cols:
    fact[c] = pd.to_numeric(fact[c], errors='coerce')
mask_valid = fact[numeric_cols].notna().any(axis=1)
fact = fact.loc[mask_valid].copy()

fact['date_key'] = fact[DATE_COL].dt.strftime('%Y%m%d').astype(int)
fact['geo_key'] = 'USA'

# Sanitize columns
fact = sanitize_columns(fact)

out_path = os.path.join(CLEAN, 'bank_aggregates_cleaned.csv')
fact.to_csv(out_path, index=False, encoding='utf-8')
print('Wrote:', out_path)


Wrote: G:\ACADEMIA\VA 5122\Final Project\phase1_cleaning_preprocessing\data\cleaned\bank_aggregates_cleaned.csv


What: Validate shape, date range, nulls, and date_key uniqueness.
Why: Catch anomalies before downstream modeling.


In [7]:
# Validation
print('Shape:', fact.shape)
print('Min/Max date_key:', fact['date_key'].min(), fact['date_key'].max())
print('Nulls (top 10):')
print(fact.isna().sum().sort_values(ascending=False).head(10))
print('Unique date_key count:', fact['date_key'].nunique())


Shape: (77, 40)
Min/Max date_key: 20000101 20190101
Nulls (top 10):
roe                            2
net_chargeoffs_loans           2
loan_loss_provisions_loans     2
unlevered_roa                  2
loan_loss_reserves_loans       2
risk_weighted_assets_assets    1
other                          1
securitization                 1
service_fees                   1
market_leverage                1
dtype: int64
Unique date_key count: 77


## Decisions & Notes

- Parsed period to `date`; if year-only, used Jan-01 fallback.
- Coerced numerics with `errors='coerce'` and documented missingness.
- Keys: `date_key` (YYYYMMDD), `geo_key='USA'`.
- Any unit normalization (percent 0–1 vs 0–100) must be noted here.
